In [40]:
USE_GROQ = True  # Set to False to use OpenAI
REQUEST_DELAY = 0.5

In [41]:
!pip install datasets

In [42]:
!pip install openai
!pip install groq

In [44]:
# Core libraries
import os
import re
import json
from groq import Groq
import openai
import json
import ast

# Data handling
import pandas as pd
from datasets import load_dataset
from collections import defaultdict


# Model & inference (if using Hugging Face transformers)
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     BitsAndBytesConfig,
#     pipeline
# )

# Evaluation (optional at this stage since you're not doing full metrics yet)
from sklearn.metrics import accuracy_score

# PyTorch (only if needed for model management or tensor ops)
import torch

from itertools import cycle

from huggingface_hub import login
login("token")


In [45]:

# List of your GROQ API keys
GROQ_API_KEYS = [
    # paid
    "TEST_KEY"
]

# Create a cycling iterator over the API keys
groq_clients = [Groq(api_key=key) for key in GROQ_API_KEYS]
groq_client_cycle = cycle(groq_clients)



In [48]:
dataset_name = "cardiffnlp/databench"
semeval_train = load_dataset(dataset_name, name="qa", split="train")
# semeval_dev = load_dataset(dataset_name, name="semeval", split="dev")

# dev_target_datasets = ['051_Pokemon', '052_Professional', '055_German']
# semeval_dev = semeval_dev.filter(lambda sample: sample['dataset'] in dev_target_datasets)

# train_target_datasets = ['051_Pokemon', '052_Professional', '055_German']


train_target_datasets = [
    "001_Forbes", "002_Titanic", "004_Taxi", "005_NYC",
    "006_London", "007_Fifa", "008_Tornados", "009_Central", "010_ECommerce",
    "011_SF", "012_Heart", "013_Roller", "015_Food",
    "016_Holiday", "017_Hacker", "018_Staff", "019_Aircraft",
    "021_Telco", "022_Airbnbs", "023_Climate", "024_Salary", "025_Data",
    "026_Predicting", "027_Supermarket", "028_Predict", "029_NYTimes", "030_Professionals",
    "031_Trustpilot", "032_Delicatessen", "033_Employee", "034_World",
    "036_US", "037_Ted", "038_Stroke", "039_Happy", "040_Speed",
    "041_Airline", "042_Predict", "043_Predict", "044_IMDb", "045_Predict",
    "046_120", "047_Bank", "048_Data",  "050_ING",
    "051_Pokemon", "052_Professional", "053_Patents", "055_German",
    "056_Emoji", "057_Spain", "058_US", "059_Second", "060_Bakery",
    "061_Disneyland", "062_Trump", "063_Influencers", "064_Clustering", "065_RFM"
]



# Not running:  "020_Real", "035_Billboard", "049_Boris", "054_Joe",

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

In [49]:
print(len(train_target_datasets))

59


In [53]:
dev_dataset_map = {}
unique_dev_datasets = list(set(semeval_train.unique("dataset")))

for dataset in train_target_datasets:
    dev_dataset_map[dataset] = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{dataset}/sample.parquet")


In [54]:
print(len(dev_dataset_map))

59


In [55]:
def clean_column_name(col_name):
    # Use regex to remove the <gx:...> part
    cleaned = re.sub(r'<gx:[^>]+>', '', col_name)
    return cleaned

for ds in dev_dataset_map.keys():
    # Get the original dataframe
    df = dev_dataset_map[ds]

    column_mapping = {col: clean_column_name(col) for col in df.columns}

    # Rename the columns in the dataframe
    df = df.rename(columns=column_mapping)

    # Store the updated dataframe back in the hashmap
    dev_dataset_map[ds] = df

In [56]:
def safe_parse_list(raw):
    import ast
    import re

    if isinstance(raw, list):
        return raw

    if isinstance(raw, str):
        try:
            # Step 1: Try literal_eval first (may work if quotes are fine)
            parsed = ast.literal_eval(raw)
            if isinstance(parsed, list):
                return parsed
        except:
            pass

        try:
            # Step 2: Manually repair unquoted or single-quoted elements
            if raw.startswith("[") and raw.endswith("]"):
                inner = raw[1:-1].strip()
                if inner and not inner.startswith(("'", '"')):
                    # Case: [Weight, Height]
                    parts = [p.strip() for p in inner.split(",")]
                    quoted = [f'"{p}"' for p in parts if p]
                    fixed = "[" + ", ".join(quoted) + "]"
                else:
                    # Case: ["What's your name?"]
                    # Escape inner quotes
                    inner = raw[1:-1]
                    fixed_inner = re.sub(r'(["\'])', r'\\\1', inner)
                    fixed = f'["{fixed_inner}"]'
                parsed = ast.literal_eval(fixed)
                if isinstance(parsed, list):
                    return parsed
        except Exception as e:
            print(f"Failed to parse used_cols string: {raw} | Reason: {e}")

    return raw # should never happen

def clean_columns_used(col_list):
    parsed = safe_parse_list(col_list)

    if not isinstance(parsed, list):
        return [str(parsed)] if parsed is not None else []

    cleaned = []
    for col in parsed:
        col = str(col)
        col = re.sub(r"<gx:[^>]+>", "", col).strip()
        cleaned.append(col)
    return cleaned

def update_columns_used(example):
    example["columns_used"] = clean_columns_used(example.get("columns_used"))
    return example

semeval_train = semeval_train.map(update_columns_used)

In [57]:
def serialize_to_kv_format(df, dropna=True):
    kv_serialized = []
    for _, row in df.iterrows():
        kv_pairs = []
        for col, val in row.items():
            if pd.isna(val) and dropna:
                continue
            if isinstance(val, str):
                val = f'"{val}"'
            kv_pairs.append(f"{col}: {val}")
        row_str = "{" + ", ".join(kv_pairs) + "}"
        kv_serialized.append(row_str)
    return kv_serialized

def build_prompt(df: pd.DataFrame, question: str, explain: bool = False, use_csv_format: bool = True) -> str:
    if use_csv_format:
        serialized_data = df.to_csv(index=False)
        data_block = f"In the following CSV formatted data:\n```csv\n{serialized_data}```"
    else:
        serialized_data = serialize_to_kv_format(df)
        data_block = f"In the following key-value formatted data:\n⁠kv\n{serialized_data}\n⁠ "

    response_format = (
        'You must answer in a single JSON with two fields:\n'
        '* "answer": your final answer based on the records.\n'
        '* "columns_used": list of relevant columns.'
    )

    prompt_body = (
        "You are an assistant tasked with answering questions asked of a given dataset in JSON format.\n"
        f"{response_format}\n"
        "Requirements:\n"
        "* Only respond with the JSON. Do not include explanations or full objects.\n"
        "* Your answer must use valid Python data types:\n"
        "  - Use `True` or `False` (capitalized) for boolean values.\n"
        "  - Use numbers as Python `int` or `float` (e.g., `3`, `3.14`).\n"
        "  - Use double-quoted Python strings for categorical values (e.g., \"USA\").\n"
        "  - Use Python lists for answers involving multiple values:\n"
        "    - For list[category], return a list of strings.\n"
        "    - For list[number], return a list of ints or floats.\n"
        "    - Ensure all inner values match the correct type.\n"
        "* Do not include explanations or justification.\n\n"
        f"{data_block}\n"
        f"USER: {question}\n"
        "ASSISTANT:"
    )

    return f"[INST]\n{prompt_body}\n[/INST]"



In [58]:
# def generate_model_response(prompt):
#     if USE_GROQ:
#         response = groq_client.chat.completions.create(
#             # model="llama3-8b-8192",
#             model="meta-llama/llama-4-scout-17b-16e-instruct",
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.7,
#             max_tokens=512,
#             top_p=1.0
#         )
#         return response.choices[0].message.content

#     else:
#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo",
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.7,
#             max_tokens=512,
#             top_p=1.0
#         )
#         return response.choices[0].message["content"]

def generate_model_response(prompt):
    """
    Generates a response using Groq (LLaMA-3) or OpenAI (GPT-3.5).
    Falls back across multiple Groq keys if needed.
    """
    if USE_GROQ:
        for attempt in range(len(groq_clients)):
            groq_client = next(groq_client_cycle)

            try:
                response = groq_client.chat.completions.create(
                    model="llama3-8b-8192",
                    # model="llama3-70b-8192",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7,
                    max_tokens=512,
                    top_p=1.0
                )
                return response.choices[0].message.content

            except Exception as e:
                print(f"[Groq Attempt {attempt + 1}] Error: {e}")
                continue

        return "All Groq API keys exhausted or rate limited."

    else:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=512,
                top_p=1.0
            )
            return response.choices[0].message["content"]

        except Exception as e:
            return f"OpenAI request failed: {e}"


In [59]:
def normalize_number(value):
    if isinstance(value, (int, float)):
        return float(value)
    if isinstance(value, str):
        return float(value.strip())
    raise ValueError(f"Expected numeric type for number, got: {type(value)}")

def normalize_category(value):
    if isinstance(value, str):
        return value.strip()
    raise ValueError(f"Expected string for category, got: {type(value)}")

def normalize_boolean(value):
    if isinstance(value, bool):
        return value
    if isinstance(value, str):
        val = value.strip().lower()
        if val in {"true", "1", "yes"}:
            return True
        elif val in {"false", "0", "no"}:
            return False
        else:
            raise ValueError(f"Unrecognized string for boolean: {value!r}")
    if isinstance(value, (int, float)):
        if value == 1:
            return True
        elif value == 0:
            return False
        else:
            raise ValueError(f"Numeric value not valid for boolean: {value}")
    raise ValueError(f"Expected bool, int, float, or string for boolean, got: {type(value)}")

In [60]:
def normalize_list_category(answer):
    """
    Normalize an answer of type list[category] into a set of cleaned strings.
    Handles both true lists and stringified list representations,
    and removes stray square brackets from individual elements.
    """
    def clean_item(x):
        x = str(x).strip()
        if x.startswith("["):
            x = x[1:]
        if x.endswith("]"):
            x = x[:-1]
        return x.strip()

    if isinstance(answer, str):
        try:
            parsed = ast.literal_eval(answer)
            if isinstance(parsed, list):
                answer = parsed
            else:
                answer = [item.strip() for item in answer.split(",") if item.strip()]
        except:
            answer = [item.strip() for item in answer.split(",") if item.strip()]

    if isinstance(answer, list):
        return set(clean_item(x) for x in answer)

    return set()  # fallback


In [61]:
def normalize_list_number(value):
    """
    Normalizes a predicted or gold value for list[number] questions.

    Expected input:
    - a string representing a list of numbers, e.g., "[2, 2, 2]"
    - OR a Python list of numbers
    - OR a stringified CSV like "2, 2, 2"

    Returns:
    - A set of floats
    """
    try:
        if isinstance(value, str):
            try:
                # Try parsing as JSON list
                value = json.loads(value)
            except json.JSONDecodeError:
                try:
                    value = ast.literal_eval(value)
                except:
                    # fallback: comma-separated
                    value = [item.strip() for item in value.split(",") if item.strip()]
        return set(float(v) for v in value)
    except Exception as e:
        print(f"normalize_list_number error: {e}")
        return None


In [62]:
def normalize_answer(value, expected_type):
    """
    Dispatches to the appropriate normalization function based on expected_type.
    """
    try:
        if expected_type == "number":
            return normalize_number(value)
        elif expected_type == "category":
            return normalize_category(value)
        elif expected_type == "boolean":
            return normalize_boolean(value)
        elif expected_type == "list[category]":
            return normalize_list_category(value)
        elif expected_type == "list[number]":
            return normalize_list_number(value)
        else:
            raise ValueError(f"Unsupported expected type: {expected_type}")
    except Exception as e:
        print(f"normalize_answer error for type '{expected_type}': {e}")
        return None


In [63]:
def normalize_columns(value):
    """
    Normalize a gold or predicted column list into a set of strings,
    preserving casing and special characters.
    """
    try:
        if isinstance(value, str):
            # Try parsing as a list
            try:
                parsed = ast.literal_eval(value)
                if isinstance(parsed, list):
                    value = parsed
                else:
                    # fallback: comma-split string
                    value = [item.strip() for item in value.split(",") if item.strip()]
            except:
                value = [item.strip() for item in value.split(",") if item.strip()]

        if isinstance(value, list):
            return set(str(x).strip() for x in value)

    except Exception as e:
        print(f"normalize_columns error: {e}")

    return set()  # fallback


In [ ]:
error_dict = {}

In [64]:
import json
import ast

def process_response(generated_text, question, expected_type, error_set):
    """
    Processes the raw LLM response to extract and normalize the answer and columns.

    Args:
        generated_text (str): Raw text output from the LLM.
        question (str): The question (used for debugging).
        expected_type (str): The expected type of the answer (e.g., boolean, number).
        error_set (set): A set to store questions that had format errors.

    Returns:
        Tuple[bool, Any, List[str]]:
            - is_error (bool): True if formatting/parsing failed.
            - norm_answer: normalized answer or None on failure.
            - norm_columns: normalized list of columns or [] on failure.
    """
    is_error = False
    norm_answer = None
    norm_columns = []

    try:
        # Step 1: Trim response
        generated_text = generated_text.strip()

        # Step 2: Try to isolate a dictionary from the output
        start = generated_text.find('{')
        end = generated_text.rfind('}') + 1
        if start == -1 or end == -1:
            raise ValueError("Could not find a JSON-like object")

        json_str = generated_text[start:end]

        # Fix lowercase true/false if needed
        json_str_fixed = json_str.replace("true", "True").replace("false", "False")

        # Step 3: Try parsing as JSON first, fallback to ast.literal_eval
        try:
            response_json = json.loads(json_str)
        except json.JSONDecodeError:
            response_json = ast.literal_eval(json_str_fixed)

        # Step 4: Ensure expected keys exist
        if "answer" not in response_json or "columns_used" not in response_json:
            raise KeyError("Missing 'answer' or 'columns_used' in response")

        raw_answer = response_json["answer"]
        raw_columns = response_json["columns_used"]

        # Step 5: Normalize both fields
        norm_answer = normalize_answer(raw_answer, expected_type)
        norm_columns = normalize_columns(raw_columns)

    except Exception as e:
        # On any failure, flag error and return safe defaults
        print(f"[process_response] Failed to parse response for question: {question}")
        print(f"Error: {e}")
        is_error = True
        error_set.add(question)
        norm_answer = None
        norm_columns = []

    # Return tuple: (was error?, normalized answer, normalized columns)
    return is_error, norm_answer, norm_columns


In [65]:
example = semeval_train[0]
question = example["question"]
dataset_id = example["dataset"]
expected_type = example["type"]
true_answer = normalize_answer(example["sample_answer"], expected_type)
true_columns = normalize_columns(example["columns_used"])

print(f"Type(true_answer) {type(true_answer)}")
print(f"Type(true_columns) {type(true_columns)}")

df = dev_dataset_map[dataset_id]
prompt = build_prompt(df, question)
# print("=== Prompt ===\n", prompt[:1000])  # Truncated for readability

model_output = generate_model_response(prompt)
# print("=== Raw Model Output ===\n", model_output)

sample_error_set = set()

isPredError, pred_answer, pred_cols = process_response(model_output, question, expected_type, sample_error_set)

print(f"\nType(pred_answer) {type(pred_answer)}")
print(f"Type(pred_columns) {type(pred_cols)}")

print(f"\nQuestion: {question}")
print(f"Is Error? : {isPredError}")
print(f"Gold Answer: {true_answer}, Predicted: {pred_answer}")
print(f"Gold Columns: {true_columns}, Predicted: {pred_cols}\n")


Type(true_answer) <class 'bool'>
Type(true_columns) <class 'set'>

Type(pred_answer) <class 'bool'>
Type(pred_columns) <class 'set'>

Question: Is the person with the highest net worth self-made?
Is Error? : False
Gold Answer: False, Predicted: True
Gold Columns: {'selfMade', 'finalWorth'}, Predicted: {'selfMade', 'finalWorth'}



In [66]:
import time

def safe_generate_response(prompt, retries=1, delay=3):
    attempts = 0
    while attempts <= retries:
        try:
            return generate_model_response(prompt)
        except Exception as e:
            print(f"[Attempt {attempts+1}] Error: {repr(e)}")
            time.sleep(delay)
            attempts += 1
    print("Failed all attempts. Returning empty response.")
    return ""


In [67]:
def evaluate_dataset(dataset_rows, dataset_name, dev_dataset_map, request_delay=1.5):
    pred_answers = []
    gold_answers = []
    pred_columns = []
    gold_columns = []
    question_types = []

    type_wise_correct = defaultdict(int)
    type_wise_total = defaultdict(int)
    column_match_count = 0
    error_set = set()
    formatting_errors_by_type = defaultdict(int)

    print(f"# Questions in {dataset_name}: {len(dataset_rows)}")

    for i, row in enumerate(dataset_rows):
        print(f"\n--- Query {i+1}/{len(dataset_rows)} ---")

        question = row["question"]
        dataset = row["dataset"]
        expected_type = row["type"]

        gold_answer = normalize_answer(row["sample_answer"], expected_type)
        gold_cols = normalize_columns(row["columns_used"])

        df = dev_dataset_map[dataset]
        prompt = build_prompt(df, question)
        # print(f"promot: {prompt}")

        response = safe_generate_response(prompt)
        time.sleep(request_delay)
        print("\n--- Raw LLM Response ---")
        print(response)

        is_error, pred_answer, pred_cols = process_response(response, question, expected_type, error_set)
        if is_error:
            formatting_errors_by_type[expected_type] += 1

        print(f"\nQuestion: {question}")
        print(f"Pred Answer: {pred_answer}, Gold Answer: {gold_answer}")
        print(f"Pred Columns: {pred_cols}, Gold Columns: {gold_cols}")

        pred_answers.append(pred_answer)
        gold_answers.append(gold_answer)
        pred_columns.append(pred_cols)
        gold_columns.append(gold_cols)
        question_types.append(expected_type)
        type_wise_total[expected_type] += 1

        correct = False
        try:
            if expected_type == "number":
                correct = abs(pred_answer - gold_answer) < 1e-3
            else:
                correct = pred_answer == gold_answer
        except:
            correct = False

        if correct:
            type_wise_correct[expected_type] += 1

        if isinstance(pred_cols, (list, set)) and set(pred_cols) == set(gold_cols):
            column_match_count += 1

    print("\n=== Answer Accuracy by Type ===")
    for qtype in type_wise_total:
        total = type_wise_total[qtype]
        correct = type_wise_correct[qtype]
        acc = correct / total if total else 0
        print(f"{qtype:15}: {acc:.2%} ({correct}/{total})")

    total = len(dataset_rows)
    col_acc = column_match_count / total if total else 0
    print(f"\n=== Column Selection Accuracy ===\n{col_acc:.2%} ({column_match_count}/{total})")

    eval_records = []
    for i in range(len(dataset_rows)):
        eval_records.append({
            "type": question_types[i],
            "gold_answer": gold_answers[i],
            "pred_answer": pred_answers[i],
            "gold_columns": gold_columns[i],
            "pred_columns": pred_columns[i],
        })

    # Final column stats
    wrong_cols = 0
    right_cols = 0
    format_errors = 0
    for i, (pred, gold) in enumerate(zip(pred_columns, gold_columns)):
        question = dataset_rows[i]["question"]

        if question in error_set:
            format_errors += 1
            continue

        if isinstance(pred, (list, set)):
            if set(pred) == set(gold):
                right_cols += 1
            else:
                wrong_cols += 1
        else:
            format_errors += 1  # fallback, shouldn't happen


    column_stats = {
        "wrong_cols": wrong_cols,
        "right_cols": right_cols,
        "format_error": format_errors,
        "total": len(dataset_rows)
    }

    return eval_records, column_stats, formatting_errors_by_type

In [68]:
def compute_metrics(eval_records, model_name, model_results):
    results = defaultdict(float)
    total = len(eval_records)
    correct_all = 0
    typewise = defaultdict(lambda: [0, 0])
    colwise = {'single': [0, 0], 'multi': [0, 0]}

    for record in eval_records:
        t = record['type'].strip().lower()
        gold_answer = record['gold_answer']
        pred_answer = record['pred_answer']
        gold_cols = set(record['gold_columns'])
        pred_cols = set(record['pred_columns'])

        try:
            if t == "number":
                answer_match = abs(pred_answer - gold_answer) < 1e-3
            else:
                answer_match = pred_answer == gold_answer
        except:
            answer_match = False

        col_match = gold_cols == pred_cols
        joint_match = answer_match and col_match
        if joint_match:
            correct_all += 1

        typewise[t][1] += 1
        if joint_match:
            typewise[t][0] += 1

        col_count = len(gold_cols)
        if col_count == 1:
            colwise['single'][1] += 1
            if joint_match:
                colwise['single'][0] += 1
        else:
            colwise['multi'][1] += 1
            if joint_match:
                colwise['multi'][0] += 1

    def get_acc(dic, key):
        correct, total = dic[key]
        return correct / total if total else 0

    results['avg'] = correct_all / total if total else 0
    results['boolean'] = get_acc(typewise, 'boolean')
    results['number'] = get_acc(typewise, 'number')
    results['category'] = get_acc(typewise, 'category')
    results['list[category]'] = get_acc(typewise, 'list[category]')
    results['list[number]'] = get_acc(typewise, 'list[number]')
    results['single col'] = get_acc(colwise, 'single')
    results['multiple cols'] = get_acc(colwise, 'multi')

    model_results[model_name] = dict(results)


In [69]:
def format_percent_and_count(val, total):
    percent = 100 * val / total if total else 0
    return f"{percent:.1f} ({val})"

In [ ]:
# # Step 1: Filter dataset
# pokemon_rows = semeval_dev.filter(lambda sample: sample["dataset"] == "051_Pokemon")

# # Step 2: Dictionary to hold results
# model_results = {}

# # Step 3: Run evaluation
# records, col_stats, formatting_errors_by_type = evaluate_dataset(
#     pokemon_rows, dataset_name="Pokemon", dev_dataset_map=dev_dataset_map
# )

# # Step 4: Compute main metrics
# compute_metrics(records, "baseline_llm", model_results)

# # Step 5a: Table 1 – main metrics
# results_df = pd.DataFrame(model_results).T.round(3)
# print("### Main Metrics Table:")
# print(results_df.to_markdown())

# # Step 5b: Table 2 – column quality summary

# col_eval_table = [{
#     "model": "baseline_llm",
#     "wrong cols": format_percent_and_count(col_stats["wrong_cols"], col_stats["total"]),
#     "right cols": format_percent_and_count(col_stats["right_cols"], col_stats["total"]),
#     "format error": format_percent_and_count(col_stats["format_error"], col_stats["total"]),
# }]

# col_df = pd.DataFrame(col_eval_table)
# print("\n### Column Quality Table:")
# print(col_df.to_markdown(index=False))

# # Step 5c: Formatting errors by type
# print("\n### Formatting Errors by Question Type:")
# for qtype, count in formatting_errors_by_type.items():
#     print(f"{qtype:15}: {count} formatting errors")


In [70]:
# model_results = {}
# column_quality_table = []
# formatting_errors_summary = defaultdict(int)

# for dataset_id in train_target_datasets:

#     dataset_rows = semeval_train.filter(lambda sample: sample["dataset"] == dataset_id)
#     print(f"\n\n### Evaluating {dataset_id} ###")

#     records, col_stats, formatting_errors_by_type = evaluate_dataset(
#         dataset_rows, dataset_name=dataset_id, dev_dataset_map=dev_dataset_map
#     )

#     compute_metrics(records, dataset_id, model_results)

#     column_quality_table.append({
#         "model": dataset_id,
#         "wrong cols": format_percent_and_count(col_stats["wrong_cols"], col_stats["total"]),
#         "right cols": format_percent_and_count(col_stats["right_cols"], col_stats["total"]),
#         "format error": format_percent_and_count(col_stats["format_error"], col_stats["total"]),
#     })

#     for qtype, count in formatting_errors_by_type.items():
#         formatting_errors_summary[qtype] += count



all_eval_records = []
model_results = {}
column_quality_table = []
formatting_errors_summary = defaultdict(int)

for dataset_id in train_target_datasets:

    dataset_rows = semeval_train.filter(lambda sample: sample["dataset"] == dataset_id)
    print(f"\n\n### Evaluating {dataset_id} ###")

    records, col_stats, formatting_errors_by_type = evaluate_dataset(
        dataset_rows, dataset_name=dataset_id, dev_dataset_map=dev_dataset_map
    )

    compute_metrics(records, dataset_id, model_results)

    all_eval_records.extend(records)

    column_quality_table.append({
        "model": dataset_id,
        "wrong cols": format_percent_and_count(col_stats["wrong_cols"], col_stats["total"]),
        "right cols": format_percent_and_count(col_stats["right_cols"], col_stats["total"]),
        "format error": format_percent_and_count(col_stats["format_error"], col_stats["total"]),
    })

    for qtype, count in formatting_errors_by_type.items():
        formatting_errors_summary[qtype] += count


Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]



### Evaluating 001_Forbes ###
# Questions in 001_Forbes: 25

--- Query 1/25 ---

--- Raw LLM Response ---
{
"answer": False,
"columns_used": ["selfMade", "finalWorth"]
}

Question: Is the person with the highest net worth self-made?
Pred Answer: False, Gold Answer: False
Pred Columns: {'selfMade', 'finalWorth'}, Gold Columns: {'selfMade', 'finalWorth'}

--- Query 2/25 ---

--- Raw LLM Response ---
{
"answer": True,
"columns_used": ["gender", "age"]
}

Question: Does the youngest billionaire identify as male?
Pred Answer: True, Gold Answer: True
Pred Columns: {'age', 'gender'}, Gold Columns: {'age', 'gender'}

--- Query 3/25 ---

--- Raw LLM Response ---
{
"answer": "Atlanta",
"columns_used": ["city", "selfMade", "country"]
}
normalize_answer error for type 'boolean': Unrecognized string for boolean: 'Atlanta'

Question: Is the city with the most billionaires in the United States?
Pred Answer: None, Gold Answer: True
Pred Columns: {'country', 'selfMade', 'city'}, Gold Columns: {'count

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]



### Evaluating 002_Titanic ###
# Questions in 002_Titanic: 20

--- Query 1/20 ---

--- Raw LLM Response ---
{
"answer": false,
"columns_used": ["Age"]
}

Question: Did any children below the age of 18 survive?
Pred Answer: False, Gold Answer: True
Pred Columns: {'Age'}, Gold Columns: {'Age', 'Survived'}

--- Query 2/20 ---

--- Raw LLM Response ---
{
"answer": True,
"columns_used": ["Fare"]
}

Question: Were there any passengers who paid a fare of more than $500?
Pred Answer: True, Gold Answer: False
Pred Columns: {'Fare'}, Gold Columns: {'Fare'}

--- Query 3/20 ---

--- Raw LLM Response ---
{"answer": False, "columns_used": ["Name"]}

Question: Is every passenger's name unique?
Pred Answer: False, Gold Answer: True
Pred Columns: {'Name'}, Gold Columns: {'Name'}

--- Query 4/20 ---

--- Raw LLM Response ---
{
    "answer": True,
    "columns_used": ["Sex", "Pclass", "Survived"]
}

Question: Were there any female passengers in the 3rd class who survived?
Pred Answer: True, Gold Answer

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

...



### Evaluating 063_Influencers ###
# Questions in 063_Influencers: 20

--- Query 1/20 ---

--- Raw LLM Response ---
{
"answer": True,
"columns_used": ["is_organization"]
}

Question: Are there any organizations in the dataset?
Pred Answer: True, Gold Answer: False
Pred Columns: {'is_organization'}, Gold Columns: {'is_organization'}

--- Query 2/20 ---

--- Raw LLM Response ---
{
"answer": True,
"columns_used": ["name", "is_organization"]
}

Question: Are there any individuals (non-organizations) in the dataset?
Pred Answer: True, Gold Answer: True
Pred Columns: {'name', 'is_organization'}, Gold Columns: {'is_organization'}

--- Query 3/20 ---

--- Raw LLM Response ---
{
"answer": True,
"columns_used": ["pic"]
}

Question: Do all entities have a picture?
Pred Answer: True, Gold Answer: True
Pred Columns: {'pic'}, Gold Columns: {'pic'}

--- Query 4/20 ---

--- Raw LLM Response ---
{
    "answer": True,
    "columns_used": ["name", "weight"]
}

Question: Are there any entities with a we

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]



### Evaluating 064_Clustering ###
# Questions in 064_Clustering: 20

--- Query 1/20 ---

--- Raw LLM Response ---
{
"answer": ["gull"],
"columns_used": ["feathers", "animal_name"]
}
normalize_answer error for type 'boolean': Expected bool, int, float, or string for boolean, got: <class 'list'>

Question: Are there any animals with feathers in the dataset?
Pred Answer: None, Gold Answer: True
Pred Columns: {'feathers', 'animal_name'}, Gold Columns: {'feathers'}

--- Query 2/20 ---

--- Raw LLM Response ---
{
"answer": true,
"columns_used": ["venomous", "animal_name"]
}

Question: Are there any venomous animals in the dataset?
Pred Answer: True, Gold Answer: True
Pred Columns: {'venomous', 'animal_name'}, Gold Columns: {'venomous'}

--- Query 3/20 ---

--- Raw LLM Response ---
{"answer": True, "columns_used": ["breathes"]}

Question: Do all animals breathe?
Pred Answer: True, Gold Answer: False
Pred Columns: {'breathes'}, Gold Columns: {'breathes'}

--- Query 4/20 ---

--- Raw LLM Resp

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]



### Evaluating 065_RFM ###
# Questions in 065_RFM: 20

--- Query 1/20 ---

--- Raw LLM Response ---
{"answer": True, "columns_used": ["Quantity"]}

Question: Are all the quantities greater than 0?
Pred Answer: True, Gold Answer: False
Pred Columns: {'Quantity'}, Gold Columns: {'Quantity'}

--- Query 2/20 ---

--- Raw LLM Response ---
{"answer": True, "columns_used": ["UnitPrice"]}

Question: Are all the unit prices greater than 0?
Pred Answer: True, Gold Answer: True
Pred Columns: {'UnitPrice'}, Gold Columns: {'UnitPrice'}

--- Query 3/20 ---

--- Raw LLM Response ---
{"answer": False, "columns_used": ["CustomerID"]}

Question: Are there any missing customer IDs?
Pred Answer: False, Gold Answer: True
Pred Columns: {'CustomerID'}, Gold Columns: {'CustomerID'}

--- Query 4/20 ---

--- Raw LLM Response ---
{
"answer": True,
"columns_used": ["Country"]
}

Question: Are there any transactions from the United Kingdom?
Pred Answer: True, Gold Answer: True
Pred Columns: {'Country'}, Gold Col

In [71]:
# # Table 1: Main Metrics Table
# results_df = pd.DataFrame(model_results).T.round(3)
# print("### Main Metrics Table:")
# print(results_df.to_markdown())

# # Table 2: Column Quality Table
# col_df = pd.DataFrame(column_quality_table)
# print("\n### Column Quality Table:")
# print(col_df.to_markdown(index=False))

# # Table 3: Formatting Errors by Type
# print("\n### Formatting Errors by Question Type:")
# for qtype, count in formatting_errors_summary.items():
#     print(f"{qtype:15}: {count} formatting errors")

# Table 1: Main Metrics Table (per dataset)
results_df = pd.DataFrame(model_results).T.round(3)
print("### Main Metrics Table:")
print(results_df.to_markdown())

# Table 2: Column Quality Table (per dataset)
col_df = pd.DataFrame(column_quality_table)
print("\n### Column Quality Table:")
print(col_df.to_markdown(index=False))

# Table 3: Formatting Errors by Question Type
print("\n### Formatting Errors by Question Type:")
for qtype, count in formatting_errors_summary.items():
    print(f"{qtype:15}: {count} formatting errors")

### Main Metrics Table:
|                   |   avg |   boolean |   number |   category |   list[category] |   list[number] |   single col |   multiple cols |
|:------------------|------:|----------:|---------:|-----------:|-----------------:|---------------:|-------------:|----------------:|
| 001_Forbes        |  0.32 |      0.6  |    0.4   |       0.4  |             0.2  |           0    |        0.429 |           0.278 |
| 002_Titanic       |  0.25 |      0.25 |    0.5   |       0.25 |             0    |           0.25 |        0.286 |           0.231 |
| 004_Taxi          |  0.1  |      0.25 |    0.25  |       0    |             0    |           0    |        0.1   |           0     |
| 005_NYC           |  0.25 |      1    |    0     |       0.25 |             0    |           0    |        0.25  |           0     |
| 006_London        |  0.2  |      0.25 |    0.5   |       0    |             0    |           0.25 |        0.2   |           0     |
| 007_Fifa          |  0.45 |  

In [72]:
# Store dataset sizes
dataset_sizes = {dataset_id: len(semeval_train.filter(lambda s: s["dataset"] == dataset_id)) for dataset_id in train_target_datasets}

def compute_weighted_global_metrics(model_results, dataset_sizes):
    weighted_sums = defaultdict(float)
    total_size = sum(dataset_sizes[ds] for ds in model_results if ds != "GLOBAL")

    for dataset_id, metrics in model_results.items():
        if dataset_id == "GLOBAL":
            continue
        weight = dataset_sizes[dataset_id]
        for k, v in metrics.items():
            weighted_sums[k] += v * weight

    return {k: weighted_sums[k] / total_size for k in weighted_sums}

# Compute and store weighted global average
global_model_results = {}
global_model_results["GLOBAL"] = compute_weighted_global_metrics(model_results, dataset_sizes)

# Display as markdown table
global_df = pd.DataFrame(global_model_results).T.round(3)
print("\n### Global Metrics Across All Datasets (Weighted):")
print(global_df.to_markdown())

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1308 [00:00<?, ? examples/s]


### Global Metrics Across All Datasets (Weighted):
|        |   avg |   boolean |   number |   category |   list[category] |   list[number] |   single col |   multiple cols |
|:-------|------:|----------:|---------:|-----------:|-----------------:|---------------:|-------------:|----------------:|
| GLOBAL | 0.267 |     0.451 |    0.314 |      0.311 |            0.139 |          0.118 |        0.326 |           0.181 |
